In [1]:
import math 
import pandas as pd
from datetime import datetime
import numpy as np
import geopandas as gpd
import folium
from shapely.geometry import Point, Polygon
import folium
import math as mt

### データ抽出

In [2]:
gad = pd.read_csv(r"C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\data\Tomakomai2211_weekend_trips.csv")

In [4]:
df = gpd.read_file(r"C:/python/data/Hokkaido_shp/Hokkaido_shp_cp932.shp")

In [5]:
mesh_df = pd.read_csv(r"C:\Users\chaki\Desktop\jupyter_file\data\2024_data\mesh_sapporo.csv")

In [6]:
df_polygon = gpd.read_file(r"C:\python\data\Hokkaido_shp/Hokkaido_shp_cp932.shp", encoding='utf-8')
df_Hokkaido = df_polygon.copy()
epsg2454 = 'EPSG:2454'
df_Hokkaido = df_Hokkaido.to_crs(epsg2454)

# 選択肢となる列名を指定
choice_column = "短縮名称"  # 実際の列名に置き換えてください


selected_choice = '札幌市'

# 選択された値を変数に代入
selected_value = df_Hokkaido[df_Hokkaido[choice_column] == selected_choice]  # 選択された値を持つ行を抽出

# 例えば、抽出された値の別の列の値を変数に代入
#df_city = selected_value.loc[:, ["city-code", 'sityouson-name']]  # another_columnは、目的の列名に置き換えてください

#print('')
#print("選択された値:", selected_choice)
#print("結果:", selected_value)
#selected_value.plot()
#df_Hokkaido.plot()
# ポリゴン内のトリップのみ抽出
def extract_by_location_o(df):
  # Destinationのgeometry付与
  geometry = [Point(xy) for xy in zip(df['start_lon'], df['start_lat'])]
  gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
  # CRS変換
  epsg2454 = 'EPSG:2454'# 北海道一部　西
  gdf = gdf.to_crs(epsg2454)
  gdf = gpd.sjoin(gdf, selected_value, how='left', predicate="within")
  gdf = gdf.drop(['index_right'], axis=1)

  # グループ化し、列yにNaNが含まれるグループを削除(内内のみ，内外や外外の移動は削除)
  c_name = '短縮名称'
  df_filtered = gdf.groupby('agent_id').filter(lambda x: x[c_name].notna().all())
  return df_filtered
    
# ポリゴン内のトリップのみ抽出
def extract_by_location_d(df):
  # Destinationのgeometry付与
  geometry = [Point(xy) for xy in zip(df['end_lon'], df['end_lat'])]
  gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
  # CRS変換
  epsg2454 = 'EPSG:2454'# 北海道一部　西
  gdf = gdf.to_crs(epsg2454)
  gdf = gpd.sjoin(gdf, selected_value, how='left', predicate="within")
  gdf = gdf.drop(['index_right'], axis=1)

  # グループ化し、列yにNaNが含まれるグループを削除(内内のみ，内外や外外の移動は削除)
  c_name = '短縮名称'
  df_filtered = gdf.groupby('agent_id').filter(lambda x: x[c_name].notna().all())
  return df_filtered    

In [7]:
a = extract_by_location_o(gad)

In [8]:
a = a.drop(columns='短縮名称')

In [9]:
b = extract_by_location_d(a)

In [10]:
b

,agent_id,dep_time,dep_time_tz,arr_time,arr_time_tz,activity_time,next_dep_time,next_dep_time_tz,trav_time,wait_time,...,modes_id,longest_distance_mode_id,age,gender,home_lon,home_lat,work_lon,work_lat,geometry,短縮名称
0,682276,07:08:00,07:08:00+09,07:12:36,07:12:36+09,08:54:24,16:07:00,16:07:00+09,00:04:36,00:00:00,...,4.0,4,60,1,141.323359,43.101715,NaN,NaN,POINT (-75309.427 -99508.674),札幌市
1,682276,16:07:00,16:07:00+09,16:11:36,16:11:36+09,NaN,NaN,NaN,00:04:36,00:00:00,...,4.0,4,60,1,141.323359,43.101715,NaN,NaN,POINT (-75427.144 -99375.841),札幌市
2,682277,08:28:00,08:28:00+09,08:42:10,08:42:10+09,01:48:50,10:31:00,10:31:00+09,00:14:10,00:00:00,...,1.0,1,60,1,141.323125,43.101163,NaN,NaN,POINT (-72037.85 -108895.688),札幌市
3,682277,10:31:00,10:31:00+09,10:50:20,10:50:20+09,00:41:40,11:32:00,11:32:00+09,00:19:20,00:01:19,...,NaN,6,60,1,141.323125,43.101163,NaN,NaN,POINT (-72983.261 -104297.864),札幌市
4,682277,11:32:00,11:32:00+09,11:33:55,11:33:55+09,01:57:05,13:31:00,13:31:00+09,00:01:55,00:00:00,...,1.0,1,60,1,141.323125,43.101163,NaN,NaN,POINT (-73150.079 -103461.005),札幌市
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4586807,1235729,22:00:00,22:00:00+09,22:10:11,22:10:11+09,NaN,NaN,NaN,00:10:11,00:00:00,...,1.0,1,20,2,141.370323,43.026791,NaN,NaN,POINT (-71691.559 -107739.807),札幌市
4586808,1235730,05:55:00,05:55:00+09,05:58:10,05:58:10+09,03:50:50,09:49:00,09:49:00+09,00:03:10,00:00:00,...,1.0,1,20,2,141.368662,43.045841,NaN,NaN,POINT (-74055.879 -105279.44),札幌市
4586809,1235730,09:49:00,09:49:00+09,09:51:43,09:51:43+09,02:47:17,12:39:00,12:39:00+09,00:02:43,00:00:00,...,4.0,4,20,2,141.368662,43.045841,NaN,NaN,POINT (-74150.457 -105235.031),札幌市
4586810,1235730,12:39:00,12:39:00+09,12:47:01,12:47:01+09,01:52:59,14:40:00,14:40:00+09,00:08:01,00:00:00,...,4.0,4,20,2,141.368662,43.045841,NaN,NaN,POINT (-74193.801 -105540.739),札幌市


In [11]:
b.to_csv(r'C:\Users\chaki\Desktop\jupyter_file\data\マイクロモビリティ\data\GAD_sapporo_Nov_weekend.csv',index=None)

### OD表作成

In [11]:
df = pd.read_csv(r'C:\python\OneDrive\jupyter_file\data\GAD_sapporo_Nov.csv')

In [6]:
df

,agent_id,dep_time,dep_time_tz,arr_time,arr_time_tz,activity_time,next_dep_time,next_dep_time_tz,trav_time,wait_time,...,home_lon,home_lat,work_lon,work_lat,geometry,短縮名称,mesh_o,mesh_d,mesh_o10,mesh_d10
0,144204,12:40:00,12:40:00+09,12:47:36,12:47:36+09,00:56:24,13:44:00,13:44:00+09,00:07:36,00:00:00,...,141.339736,43.049724,NaN,NaN,POINT (-67735.39348777923 -102497.67833707939),札幌市,64414257,64414383,6441425723,6441438324
1,144204,13:44:00,13:44:00+09,13:47:14,13:47:14+09,05:05:46,18:53:00,18:53:00+09,00:03:14,00:00:00,...,141.339736,43.049724,NaN,NaN,POINT (-69350.37182218577 -102542.80341825885),札幌市,64414383,64414381,6441438324,6441438125
2,144204,18:53:00,18:53:00+09,18:59:49,18:59:49+09,01:48:11,20:48:00,20:48:00+09,00:06:49,00:00:00,...,141.339736,43.049724,NaN,NaN,POINT (-74078.09838498705 -105034.38095756902),札幌市,64414381,64414267,6441438125,6441426712
3,144204,20:48:00,20:48:00+09,20:51:58,20:51:58+09,NaN,NaN,NaN,00:03:58,00:00:00,...,141.339736,43.049724,NaN,NaN,POINT (-74156.72560790835 -105165.7430017972),札幌市,64414267,64414257,6441426712,6441425723
4,144206,12:47:00,12:47:00+09,12:50:54,12:50:54+09,03:39:06,16:30:00,16:30:00+09,00:03:54,00:00:00,...,141.339776,43.049800,NaN,NaN,POINT (-72914.13965366741 -104294.51757631708),札幌市,64414257,64414268,6441425723,6441426824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3055048,1224979,09:23:00,09:23:00+09,09:28:25,09:28:25+09,NaN,NaN,NaN,00:05:25,00:00:00,...,141.389450,43.048492,NaN,NaN,POINT (-70108.02239412944 -105345.33933212447),札幌市,64414342,64414351,6441434212,6441435123
3055049,1224982,08:32:00,08:32:00+09,08:43:49,08:43:49+09,11:40:11,20:24:00,20:24:00+09,00:11:49,00:00:00,...,141.362429,43.057722,141.421152,43.064851,POINT (-67507.32223912752 -103554.17962979552),札幌市,64414268,64414373,6441426825,6441437324
3055050,1224982,20:24:00,20:24:00+09,20:31:48,20:31:48+09,NaN,NaN,NaN,00:07:48,00:00:00,...,141.362429,43.057722,141.421152,43.064851,POINT (-72298.57407427355 -104297.13810695428),札幌市,64414373,64414268,6441437324,6441426825
3055051,1224986,07:35:00,07:35:00+09,08:01:18,08:01:18+09,11:39:42,19:41:00,19:41:00+09,00:26:18,00:00:00,...,141.388101,43.035165,141.279285,43.096634,POINT (-79021.21437891446 -99899.61872575017),札幌市,64414341,64415212,6441434111,6441521222


In [4]:
def grid5th(lon, lat): 
    import math as mt# 5次メッシュ(10桁) 8分割x10分割x2分割x2分割=320分割
    return (int(mt.floor(lat*480       / 320)) * 100000000 + int(mt.floor((lon-100)*320       / 320)) * 1000000 +    # 1次メッシュ
            int(mt.floor(lat*480 % 320 / 40))  * 100000    + int(mt.floor((lon-100)*320 % 320 / 40))  * 10000   +    # 2次メッシュ
            int(mt.floor(lat*480 % 40  / 4))   * 1000      + int(mt.floor((lon-100)*320 % 40  / 4))   * 100     +    # 3次メッシュ
            int(mt.floor(lat*480 % 4   / 2))   * 10        + int(mt.floor((lon-100)*320 % 4   / 2))   * 1       +10+    # 4次メッシュ
            int(mt.floor(lat*480)) % 2         * 2        + int(mt.floor((lon-100)*320)) % 2                  + 1) # 5次メッシュ
mesh_list = []
for i in range(len(df)):
    mesh = grid5th(df['end_lon'][i],df['end_lat'][i])
    mesh_list.append(mesh)
df['mesh_d10'] = mesh_list

In [3]:
mesh_sapporo = pd.read_csv(r"C:\python\OneDrive\jupyter_file\data\mesh_sapporo.csv")
mesh_sapporo = mesh_sapporo.drop(columns='trip')

In [12]:
df_temp = df.copy()

In [13]:
mesh_df = pd.read_csv(r"C:\python\OneDrive\jupyter_file\data\mesh_sapporo.csv")

In [14]:
import math as mt
def grid3rd(lon, lat):  # 3次メッシュ(8桁) 8分割x10分割=80分割
    return (int(mt.floor(lat*120      / 80)) * 1000000 + int(mt.floor((lon-100))             ) * 10000 +  # 1次メッシュ
            int(mt.floor(lat*120 % 80 / 10)) * 1000    + int(mt.floor((lon-100)*80 % 80 / 10)) * 100 +    # 2次メッシュ
            int(mt.floor(lat*120 % 10))      * 10      + int(mt.floor((lon-100)*80)) % 10               ) 
mesho_list = []
for i in range(len(df)):
    mesh = grid3rd(df['start_lon'][i],df['start_lat'][i])
    mesho_list.append(mesh)
df['mesh_o'] = mesho_list

meshd_list = []
for i in range(len(df)):
    mesh = grid3rd(df['start_lon'][i],df['start_lat'][i])
    meshd_list.append(mesh)
df['mesh_d'] = meshd_list
df_temp = df.copy()

In [15]:
mesh_id = list(set(mesh_df['code']))
mesh_id.sort()
# データフレームの作成
od = pd.DataFrame(index=mesh_id, columns=mesh_id)
# すべてのセルに0を設定
od = od.fillna(0)
for i in range(len(od)):
    od_index = od.index[i]
    temp_index = df_temp[df_temp['mesh_o'].isin([od_index])]
    if temp_index.empty:
        a = 0
    else:
        for j in range(len(od)):
            od_columns = od.columns[j]
            temp_columns = temp_index[temp_index['mesh_d'].isin([od_columns])]
            od.iloc[i,j] = len(temp_columns)

od['index_Sum'] = od.sum(axis=1)
od.loc['Column_Sum'] = od.sum()
od.to_csv(r"C:\Users\chaki\Desktop\jupyter_file\data\GAD_od_Nov.csv",index=None)

C:\Users\chaki\AppData\Local\Temp\ipykernel_1192\2338991883.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  od = od.fillna(0)
C:\Users\chaki\AppData\Local\Temp\ipykernel_1192\2338991883.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  od['index_Sum'] = od.sum(axis=1)


In [79]:
od = pd.read_csv(r'C:\python\OneDrive\jupyter_file\data\GAD_od_data.csv',index_col=0)

C:\Users\chaki\AppData\Local\Temp\ipykernel_22656\2238886367.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  od = pd.read_csv(r'C:\python\OneDrive\jupyter_file\data\GAD_od_data.csv',index_col=0)


In [86]:
#mesh_dfのtrip数の抽出
od_data = []
for i in range(len(od)-1):
    a = od.iloc[i,i]
    od_data.append(a)
mesh_df['trip'] = od_data

In [98]:
import ast
import folium
trip_data = pd.read_csv(r"C:\python\OneDrive\jupyter_file\data\sapporo_itrip_data.csv",index_col = 0)
#mesh_df['mesh'] = mesh_df['mesh'].apply(ast.literal_eval)

# foliumによる確認

# 地図の中心とズームレベルの設定
m = folium.Map(location=[43.068661,141.350755], zoom_start=11)  # 東京駅付近
max_value = mesh_df['trip'].max()
layer_1 = folium.FeatureGroup('sapporo').add_to(m)
# layer_1(df_sapporo)
for index, row in mesh_df.iterrows():
    opacity = row['trip'] / mesh_df['trip'].max()*2
    
    folium.Polygon(locations=row['mesh'], color='white1', fill=True, fill_color='blue', fill_opacity=opacity,tooltip=row['trip']).add_to(layer_1)

# レイヤーをマップに追加
layer_1.add_to(m)

folium.LayerControl().add_to(m)
m.save(r"C:\python\OneDrive\jupyter_file\data\GAD_sapporo_heatmap.html")
# 地図の保存

m